# 🚀 Hierarchical Subchat System - Kaggle GPU Testing

## 📋 Setup Checklist (Do Once):

### 1️⃣ **Add Kaggle Secrets** (Most Important!)
Go to: **https://www.kaggle.com/settings** → Add-ons → Secrets

Add these two secrets:
- **`GROQ_API_KEY`** = Your Groq API key (for query decomposition)
- **`GITHUB_TOKEN`** = Your GitHub personal access token (for pushing results)

### 2️⃣ **Enable Internet in This Notebook**
- Click "⚙️ Settings" (top right)
- Turn ON **"Internet"** toggle
- Click "Save"

### 3️⃣ **Make Sure This Notebook is PRIVATE**
- Never share secrets in public notebooks!

---

## ▶️ Run Order:
1. **Cells 2-7**: Load secrets and libraries
2. **Cell 8**: Set environment variables (LLM_BACKEND=vllm)
3. **Cell 9**: Load vLLM model on Kaggle GPUs (Qwen-3 14B AWQ) - **Takes 2-3 minutes**
4. **Cells 15-17**: Clone repo, configure git, run test log push
5. **Cell 21**: Integrate vLLM with backend (registers model globally)
6. **Cell 23**: Test vLLM integration
7. **Run your actual tests**: Execute test scripts in backend/dataset/
8. **Push results**: Use git_commit_and_push() function to sync to GitHub

---

## 🎯 What This Notebook Does:
- ✅ Loads vLLM model (Qwen-3 14B) on Kaggle's 2x GPUs with AWQ quantization
- ✅ Integrates vLLM with your backend via `VLLMClient` singleton
- ✅ Runs hierarchical subchat tests using GPU-accelerated inference
- ✅ Generates performance logs for buffer size analysis
- ✅ Automatically syncs results back to your GitHub repo

---

## 🔧 Technical Details:
- **vLLM Config**: Tensor parallelism across both GPUs, 91% memory utilization
- **Backend Mode**: `LLM_BACKEND=vllm` (set in cell 8)
- **Model**: Qwen-3 14B AWQ (5120 max tokens, prefix caching enabled)
- **No .env needed**: Secrets loaded from Kaggle environment
- **Git Workflow**: Clone → Test → Push results to `kaggle-run` branch

In [13]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/qwen-3/transformers/32b-awq/1/model.safetensors.index.json
/kaggle/input/qwen-3/transformers/32b-awq/1/model-00003-of-00004.safetensors
/kaggle/input/qwen-3/transformers/32b-awq/1/config.json
/kaggle/input/qwen-3/transformers/32b-awq/1/merges.txt
/kaggle/input/qwen-3/transformers/32b-awq/1/LICENSE
/kaggle/input/qwen-3/transformers/32b-awq/1/model-00001-of-00004.safetensors
/kaggle/input/qwen-3/transformers/32b-awq/1/README.md
/kaggle/input/qwen-3/transformers/32b-awq/1/tokenizer.json
/kaggle/input/qwen-3/transformers/32b-awq/1/vocab.json
/kaggle/input/qwen-3/transformers/32b-awq/1/tokenizer_config.json
/kaggle/input/qwen-3/transformers/32b-awq/1/model-00004-of-00004.safetensors
/kaggle/input/qwen-3/transformers/32b-awq/1/model-00002-of-00004.safetensors
/kaggle/input/qwen-3/transformers/32b-awq/1/generation_config.json
/kaggle/input/qwen-3/transformers/14b-awq/1/model.safetensors.index.json
/kaggle/input/qwen-3/transformers/14b-awq/1/config.json
/kaggle/input/qwen-3/trans

# CHECKING OUT MY GPU WORKINGW

*** repo: https://github.com/moonmehedi/Subchat-Trees-A-Scalable-Architecture-for-Multi-Threaded-Dialogue-and-Context-Isolation-in-LLM ***


In [14]:
! uv pip uninstall -q --system 'tensorflow'
! uv pip install -q --system  'vllm' 'triton' 'logits-processor-zoo' 'numpy<2'

In [15]:
import os
import re
import logging
from pathlib import Path
import pickle
import json
import joblib
import shutil
import glob
from tqdm.auto import tqdm
import warnings

import numpy as np
import pandas as pd



# For Qwen
import torch
import vllm
from logits_processor_zoo.vllm import MultipleChoiceLogitsProcessor


In [16]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("GITHUB_TOKEN")
secret_value_1 = user_secrets.get_secret("GROQ_API_KEY")
secret_value_2 = user_secrets.get_secret("HuggingFACEHUB_access_token")
secret_value_3 = user_secrets.get_secret("LANGCHAIN_API_KEY")

# ✅ IMPORTANT: Set them in os.environ so other code can access them
os.environ["GITHUB_TOKEN"] = secret_value_0
os.environ["GROQ_API_KEY"] = secret_value_1
os.environ["HuggingFACEHUB_access_token"] = secret_value_2
os.environ["LANGCHAIN_API_KEY"] = secret_value_3
os.environ["LLM_BACKEND"] = "vllm"

# Print the tokens (first 4 and last 4 characters for security)
print("="*60)
print("🔐 SECRETS LOADED AND SET IN ENVIRONMENT")
print("="*60)
print(f"✅ GITHUB_TOKEN: {secret_value_0[:4]}...{secret_value_0[-4:]}")
print(f"✅ GROQ_API_KEY: {secret_value_1[:4]}...{secret_value_1[-4:]}")
print(f"✅ HuggingFACEHUB_access_token: {secret_value_2[:4]}...{secret_value_2[-4:]}")
print(f"✅ LANGCHAIN_API_KEY: {secret_value_3[:4]}...{secret_value_3[-4:]}")
print(f"✅ LLM_BACKEND: vllm")
print("="*60)

🔐 SECRETS LOADED AND SET IN ENVIRONMENT
✅ GITHUB_TOKEN: gith...tWfg
✅ GROQ_API_KEY: gsk_...l6gr
✅ HuggingFACEHUB_access_token: hf_E...GaQC
✅ LANGCHAIN_API_KEY: lsv2...ea2f
✅ LLM_BACKEND: vllm


In [17]:
# vLLM V1 does not currently accept logits processor so we need to disable it
# https://docs.vllm.ai/en/latest/getting_started/v1_user_guide.html#deprecated-features
os.environ["VLLM_USE_V1"] = "0"

#model_path = "/kaggle/input/qwen2.5-coder/transformers/32b-instruct-awq/1"
# Use 14B model (32B causes linker failures with libcuda on Kaggle T4 GPUs)
model_path = "/kaggle/input/qwen-3/transformers/14b-awq/1"
llm = vllm.LLM(
    model_path,
    quantization='awq',
    tensor_parallel_size=torch.cuda.device_count(),
    gpu_memory_utilization=0.91,
    trust_remote_code=True,
    dtype="half",
    enforce_eager=True,
    max_model_len=5120,
    disable_log_stats=True,
    enable_prefix_caching=True
)

INFO 12-13 17:22:16 [utils.py:253] non-default args: {'trust_remote_code': True, 'dtype': 'half', 'seed': None, 'max_model_len': 5120, 'tensor_parallel_size': 2, 'enable_prefix_caching': True, 'gpu_memory_utilization': 0.91, 'disable_log_stats': True, 'quantization': 'awq', 'enforce_eager': True, 'model': '/kaggle/input/qwen-3/transformers/32b-awq/1'}


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


INFO 12-13 17:22:16 [model.py:637] Resolved architecture: Qwen3ForCausalLM
INFO 12-13 17:22:16 [model.py:1750] Using max model len 5120
INFO 12-13 17:22:16 [model.py:1750] Using max model len 5120
INFO 12-13 17:22:16 [scheduler.py:228] Chunked prefill is enabled with max_num_batched_tokens=8192.
WARNING 12-13 17:22:16 [vllm.py:601] Enforce eager set, overriding optimization level to -O0
INFO 12-13 17:22:16 [vllm.py:707] Cudagraph is disabled under eager mode
INFO 12-13 17:22:16 [scheduler.py:228] Chunked prefill is enabled with max_num_batched_tokens=8192.
WARNING 12-13 17:22:16 [vllm.py:601] Enforce eager set, overriding optimization level to -O0
INFO 12-13 17:22:16 [vllm.py:707] Cudagraph is disabled under eager mode


(EngineCore_DP0 pid=1743) INFO 12-13 17:22:26 [core.py:93] Initializing a V1 LLM engine (v0.12.0) with config: model='/kaggle/input/qwen-3/transformers/32b-awq/1', speculative_config=None, tokenizer='/kaggle/input/qwen-3/transformers/32b-awq/1', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=5120, download_dir=None, load_format=auto, tensor_parallel_size=2, pipeline_parallel_size=1, data_parallel_size=1, disable_custom_all_reduce=False, quantization=awq, enforce_eager=True, kv_cache_dtype=auto, device_config=cuda, structured_outputs_config=StructuredOutputsConfig(backend='auto', disable_fallback=False, disable_any_whitespace=False, disable_additional_properties=False, reasoning_parser='', reasoning_parser_plugin='', enable_in_reasoning=False), observability_config=ObservabilityConfig(show_hidden_metrics_for_version=None, otlp_traces_endpoint=None, collect_detailed_traces=None, kv_cache_met

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:07<00:23,  7.85s/it]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:07<00:23,  7.85s/it]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:21<00:23, 11.52s/it]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:21<00:23, 11.52s/it]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:36<00:12, 12.82s/it]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:36<00:12, 12.82s/it]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:48<00:00, 12.43s/it]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:48<00:00, 12.03s/it]
(Worker_TP0 pid=1754) 
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:48<00:00, 12.43s/it]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:48<00:00, 12.03s/it]
(Worker_TP0 pid=1754) 


(Worker_TP0 pid=1754) INFO 12-13 17:23:25 [default_loader.py:308] Loading weights took 48.24 seconds
(Worker_TP0 pid=1754) INFO 12-13 17:23:25 [gpu_model_runner.py:3549] Model loading took 9.0570 GiB memory and 48.815444 seconds
(Worker_TP0 pid=1754) INFO 12-13 17:23:25 [gpu_model_runner.py:3549] Model loading took 9.0570 GiB memory and 48.815444 seconds
(Worker_TP0 pid=1754) INFO 12-13 17:23:53 [gpu_worker.py:359] Available KV cache memory: 2.84 GiB
(Worker_TP0 pid=1754) INFO 12-13 17:23:53 [gpu_worker.py:359] Available KV cache memory: 2.84 GiB
(EngineCore_DP0 pid=1743) INFO 12-13 17:23:54 [kv_cache_utils.py:1286] GPU KV cache size: 23,264 tokens
(EngineCore_DP0 pid=1743) INFO 12-13 17:23:54 [kv_cache_utils.py:1291] Maximum concurrency for 5,120 tokens per request: 4.54x
(Worker_TP0 pid=1754) INFO 12-13 17:23:54 [kernel_warmup.py:65] Warming up FlashInfer attention.
(Worker_TP1 pid=1755) INFO 12-13 17:23:54 [kernel_warmup.py:65] Warming up FlashInfer attention.
(EngineCore_DP0 pid=17

(EngineCore_DP0 pid=1743) Process EngineCore_DP0:
(EngineCore_DP0 pid=1743) Traceback (most recent call last):
(EngineCore_DP0 pid=1743)   File "/usr/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
(EngineCore_DP0 pid=1743)     self.run()
(EngineCore_DP0 pid=1743)   File "/usr/lib/python3.11/multiprocessing/process.py", line 108, in run
(EngineCore_DP0 pid=1743)     self._target(*self._args, **self._kwargs)
(EngineCore_DP0 pid=1743)   File "/usr/local/lib/python3.11/dist-packages/vllm/v1/engine/core.py", line 847, in run_engine_core
(EngineCore_DP0 pid=1743)     raise e
(EngineCore_DP0 pid=1743)   File "/usr/local/lib/python3.11/dist-packages/vllm/v1/engine/core.py", line 834, in run_engine_core
(EngineCore_DP0 pid=1743)     engine_core = EngineCoreProc(*args, **kwargs)
(EngineCore_DP0 pid=1743)                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
(EngineCore_DP0 pid=1743)   File "/usr/local/lib/python3.11/dist-packages/vllm/v1/engine/core.py", line 610, in __init__
(E

RuntimeError: Engine core initialization failed. See root cause above. Failed core proc(s): {}

In [ ]:
from vllm import SamplingParams

def stream_generate(llm, prompt):
    sampling_params = SamplingParams(
        temperature=0.2,
        top_p=0.9,
        max_tokens=512,
    )

    for output in llm.generate(
        [prompt],
        sampling_params,
        
    ):
        yield output.outputs[0].text


# ── Usage ────────────────────────────────────────────────
prompt = """You are a helpful assistant.
User: Explain tensor parallelism in simple terms.
Assistant:"""

for token in stream_generate(llm, prompt):
    print(token, end="", flush=True)


NameError: name 'llm' is not defined

In [ ]:
prompt = """what is quantum computing?"""

for token in stream_generate(llm, prompt):
    print(token, end="", flush=True)

In [ ]:
print('h')

h


# test github

In [ ]:
# Load secrets from Kaggle's secure environment
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()

print("="*60)
print("🔐 LOADING SECRETS FROM KAGGLE")
print("="*60)

# Try to load GROQ_API_KEY
try:
    GROQ_API_KEY = user_secrets.get_secret("GROQ_API_KEY")
    os.environ["GROQ_API_KEY"] = GROQ_API_KEY
    print("✅ GROQ_API_KEY loaded successfully")
    print(f"   Key length: {len(GROQ_API_KEY)} characters")
except Exception as e:
    print(f"⚠️  GROQ_API_KEY not found: {e}")
    print("   Add it in Kaggle Settings → Secrets")
    GROQ_API_KEY = None

# Try to load GITHUB_TOKEN
try:
    GITHUB_TOKEN = user_secrets.get_secret("GITHUB_TOKEN")
    os.environ["GITHUB_TOKEN"] = GITHUB_TOKEN
    print("✅ GITHUB_TOKEN loaded successfully")
    print(f"   Token length: {len(GITHUB_TOKEN)} characters")
except Exception as e:
    print(f"⚠️  GITHUB_TOKEN not found: {e}")
    print("   Add it in Kaggle Settings → Secrets")
    GITHUB_TOKEN = None

# Set LLM backend to use vLLM (local model on Kaggle GPU)
os.environ["LLM_BACKEND"] = "vllm"  # We'll use the vLLM model loaded above
print("\n✅ LLM_BACKEND set to 'vllm' (using Kaggle GPU)")

print("="*60)

🔐 LOADING SECRETS FROM KAGGLE
✅ GROQ_API_KEY loaded successfully
   Key length: 56 characters
✅ GITHUB_TOKEN loaded successfully
   Token length: 93 characters

✅ LLM_BACKEND set to 'vllm' (using Kaggle GPU)
✅ GITHUB_TOKEN loaded successfully
   Token length: 93 characters

✅ LLM_BACKEND set to 'vllm' (using Kaggle GPU)


In [ ]:
# Check GPU availability and configuration
import torch

print("="*60)
print("🔍 ENVIRONMENT CHECK")
print("="*60)
print(f"✅ PyTorch version: {torch.__version__}")
print(f"✅ CUDA available: {torch.cuda.is_available()}")
print(f"✅ CUDA version: {torch.version.cuda}")
print(f"✅ Number of GPUs: {torch.cuda.device_count()}")

if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(f"\n🎮 GPU {i}: {torch.cuda.get_device_name(i)}")
        print(f"   Memory: {torch.cuda.get_device_properties(i).total_memory / 1024**3:.2f} GB")

print(f"\n✅ Current working directory: {os.getcwd()}")
print("="*60)

🔍 ENVIRONMENT CHECK
✅ PyTorch version: 2.6.0+cu124
✅ CUDA available: False
✅ CUDA version: 12.4
✅ Number of GPUs: 0

✅ Current working directory: /kaggle/working


In [ ]:
# Clone the kaggle-run branch from GitHub (PUBLIC READ - no auth needed)
import subprocess

REPO_URL = "https://github.com/moonmehedi/Subchat-Trees-A-Scalable-Architecture-for-Multi-Threaded-Dialogue-and-Context-Isolation-in-LLM.git"
REPO_DIR = "Subchat-Trees"
BRANCH = "kaggle-run"

print("="*60)
print("📥 CLONING REPOSITORY")
print("="*60)

# Remove existing directory if present
if os.path.exists(REPO_DIR):
    print(f"⚠️  Removing existing {REPO_DIR} directory...")
    shutil.rmtree(REPO_DIR)

# Clone the specific branch (no authentication needed for public repos)
# Skip LFS files to avoid bandwidth quota issues
print(f"🔄 Cloning {BRANCH} branch (skipping LFS files)...")
print("   No authentication required for cloning (public repo)")

# Set environment variable to skip LFS files
clone_env = os.environ.copy()
clone_env["GIT_LFS_SKIP_SMUDGE"] = "1"

result = subprocess.run(
    ["git", "clone", "-b", BRANCH, "--single-branch", REPO_URL, REPO_DIR],
    capture_output=True,
    text=True,
    env=clone_env
)

if result.returncode == 0:
    print(f"✅ Successfully cloned {BRANCH} branch!")
    print(f"📂 Repository location: {os.path.abspath(REPO_DIR)}")
    
    # List key directories to verify
    print("\n📁 Key directories found:")
    key_dirs = ["backend", "backend/src", "backend/dataset"]
    for dir_path in key_dirs:
        full_path = os.path.join(REPO_DIR, dir_path)
        if os.path.exists(full_path):
            print(f"   ✅ {dir_path}")
        else:
            print(f"   ❌ {dir_path} (not found)")
else:
    print(f"❌ Clone failed: {result.stderr}")
    
print("="*60)

📥 CLONING REPOSITORY
⚠️  Removing existing Subchat-Trees directory...
🔄 Cloning kaggle-run branch (skipping LFS files)...
   No authentication required for cloning (public repo)
✅ Successfully cloned kaggle-run branch!
📂 Repository location: /kaggle/working/Subchat-Trees

📁 Key directories found:
   ✅ backend
   ✅ backend/src
   ✅ backend/dataset
✅ Successfully cloned kaggle-run branch!
📂 Repository location: /kaggle/working/Subchat-Trees

📁 Key directories found:
   ✅ backend
   ✅ backend/src
   ✅ backend/dataset


In [ ]:
# Configure git identity
os.chdir(REPO_DIR)

print("="*60)
print("⚙️  CONFIGURING GIT")
print("="*60)

!git config user.name "moonmehedi"
!git config user.email "the.mehedi.hasan.moon@gmail.com"

print("✅ Git identity configured!")
print(f"   User: moonmehedi")
print(f"   Email: the.mehedi.hasan.moon@gmail.com")

# Verify current branch
branch_result = subprocess.run(["git", "branch", "--show-current"], capture_output=True, text=True)
print(f"\n✅ Current branch: {branch_result.stdout.strip()}")
print("="*60)

os.chdir("..")  # Return to parent directory

⚙️  CONFIGURING GIT
✅ Git identity configured!
   User: moonmehedi
   Email: the.mehedi.hasan.moon@gmail.com

✅ Current branch: kaggle-run
✅ Git identity configured!
   User: moonmehedi
   Email: the.mehedi.hasan.moon@gmail.com

✅ Current branch: kaggle-run


In [ ]:
def create_test_log(log_dir="kaggle_logs", log_file="connection_test.log"):
    """Create a detailed test log with GPU and environment info"""
    from datetime import datetime
    
    os.makedirs(log_dir, exist_ok=True)
    log_path = os.path.join(log_dir, log_file)
    current_time = datetime.now()
    
    with open(log_path, "w") as f:
        f.write("="*60 + "\n")
        f.write("🔬 KAGGLE GPU TEST RUN - CONNECTION VERIFICATION\n")
        f.write("="*60 + "\n\n")
        
        f.write(f"📅 Test Date: {current_time.strftime('%Y-%m-%d')}\n")
        f.write(f"⏰ Test Time: {current_time.strftime('%H:%M:%S UTC')}\n")
        f.write(f"📍 Timestamp: {pd.Timestamp.now()}\n\n")
        
        f.write("="*60 + "\n")
        f.write("🎮 GPU CONFIGURATION\n")
        f.write("="*60 + "\n")
        f.write(f"GPU Count: {torch.cuda.device_count()}\n")
        
        if torch.cuda.is_available():
            for i in range(torch.cuda.device_count()):
                f.write(f"\nGPU {i}:\n")
                f.write(f"  - Name: {torch.cuda.get_device_name(i)}\n")
                f.write(f"  - Memory: {torch.cuda.get_device_properties(i).total_memory / 1024**3:.2f} GB\n")
        else:
            f.write("⚠️  No GPU detected\n")
        
        f.write("\n" + "="*60 + "\n")
        f.write("📊 ENVIRONMENT INFO\n")
        f.write("="*60 + "\n")
        f.write(f"PyTorch Version: {torch.__version__}\n")
        f.write(f"CUDA Available: {torch.cuda.is_available()}\n")
        f.write(f"Working Directory: {os.getcwd()}\n")
        
        f.write("\n" + "="*60 + "\n")
        f.write("✅ TEST STATUS: SUCCESS\n")
        f.write("="*60 + "\n")
        f.write(f"\nThis log was generated from Kaggle notebook\n")
        f.write(f"Push attempt at: {current_time.isoformat()}\n")
    
    return log_path, current_time


def git_commit_and_push(file_path, commit_message, branch="kaggle-run"):
    """Commit a file and push to GitHub"""
    import subprocess
    
    # Add file
    add_result = subprocess.run(["git", "add", file_path], capture_output=True, text=True)
    if add_result.returncode != 0:
        return False, f"Git add failed: {add_result.stderr}"
    
    # Commit
    commit_result = subprocess.run(["git", "commit", "-m", commit_message], capture_output=True, text=True)
    if commit_result.returncode != 0:
        return False, f"Git commit failed: {commit_result.stderr}"
    
    # Push with token
    if "GITHUB_TOKEN" not in os.environ:
        return False, "GITHUB_TOKEN not found in environment"
    
    repo_url_with_token = f"https://{os.environ['GITHUB_TOKEN']}@github.com/moonmehedi/Subchat-Trees-A-Scalable-Architecture-for-Multi-Threaded-Dialogue-and-Context-Isolation-in-LLM.git"
    
    # Set remote URL
    subprocess.run(["git", "remote", "set-url", "origin", repo_url_with_token], capture_output=True)
    
    # Push
    push_result = subprocess.run(["git", "push", "origin", branch], capture_output=True, text=True)
    
    if push_result.returncode == 0:
        return True, "Push successful"
    else:
        return False, f"Push failed: {push_result.stderr}"


# Main execution
print("="*60)
print("🧪 TESTING GIT PUSH CAPABILITY")
print("="*60)

try:
    # Change to repo directory
    os.chdir(REPO_DIR)
    
    # Create test log
    log_path, timestamp = create_test_log()
    print(f"✅ Created detailed test log: {log_path}")
    
    # Commit and push
    commit_msg = f"Test: Kaggle GPU verification - {timestamp.strftime('%Y-%m-%d %H:%M:%S')}"
    success, message = git_commit_and_push(log_path, commit_msg, BRANCH)
    
    if success:
        print("\n✅ Successfully pushed to GitHub!")
        print(f"   📁 Check: {log_path}")
        print(f"   📅 Pushed at: {timestamp.strftime('%Y-%m-%d %H:%M:%S')}")
        print("   💡 Pull on your local machine to verify sync")
    else:
        print(f"\n❌ {message}")
        
except Exception as e:
    print(f"\n❌ Error: {e}")
    import traceback
    traceback.print_exc()
finally:
    # Always return to parent directory
    os.chdir("..")

print("="*60)

🧪 TESTING GIT PUSH CAPABILITY
✅ Created detailed test log: kaggle_logs/connection_test.log
[kaggle-run 7a97502] Test: Kaggle GPU verification - 2025-12-13 16:32:42
 1 file changed, 27 insertions(+)
 create mode 100644 kaggle_logs/connection_test.log
[kaggle-run 7a97502] Test: Kaggle GPU verification - 2025-12-13 16:32:42
 1 file changed, 27 insertions(+)
 create mode 100644 kaggle_logs/connection_test.log

🔄 Pushing to GitHub...

🔄 Pushing to GitHub...
✅ Successfully pushed to GitHub!
   📁 Check: kaggle_logs/connection_test.log
   📅 Pushed at: 2025-12-13 16:32:42
   💡 Pull on your local machine to verify sync
✅ Successfully pushed to GitHub!
   📁 Check: kaggle_logs/connection_test.log
   📅 Pushed at: 2025-12-13 16:32:42
   💡 Pull on your local machine to verify sync


# 🔗 Step 5: Integrate vLLM with Backend

**This connects the loaded vLLM model to your backend code**

In [ ]:
# Register the vLLM model with the backend
import sys
sys.path.insert(0, os.path.join(REPO_DIR, "backend"))

from src.services.vllm_client import VLLMClient

print("="*60)
print("🔗 INTEGRATING vLLM WITH BACKEND")
print("="*60)

# Register the globally loaded vLLM model
VLLMClient.set_model(llm)

print("✅ vLLM model is now available to backend services")
print(f"   Model: {model_path}")
print(f"   GPUs: {torch.cuda.device_count()}")
print(f"   Backend will use LLM_BACKEND={os.getenv('LLM_BACKEND')}")
print("="*60)

# 🧪 Step 6: Test vLLM Integration

**Quick test to verify backend can use vLLM on Kaggle GPU**

In [ ]:
# Test the backend with vLLM
from src.services.simple_llm import SimpleLLMClient
from src.models.tree import TreeNode, LocalBuffer

print("="*60)
print("🧪 TESTING BACKEND WITH vLLM")
print("="*60)

# Create a simple test
llm_client = SimpleLLMClient()

# Create a test node with buffer
buffer = LocalBuffer(buffer_size=5)
root = TreeNode(id="test", title="Test Conversation", buffer=buffer)

# Add a message to buffer
buffer.add_message("user", "Hello, test message")

# Test generation
print("\n📝 Testing response generation...")
response = llm_client.generate_response(root, "What is 2+2?")

print(f"\n✅ Response: {response}")
print(f"📊 Token usage: {llm_client.get_last_usage()}")
print("\n" + "="*60)
print("✅ Backend integration successful!")
print("="*60)

# testing ends